In [ ]:
# -*- coding: utf-8 -*-
import requests
import json
import pandas as pd

In [ ]:
from datetime import datetime

In [ ]:
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'

# 기준이 되는 오늘 날짜
b_date = ''.join(filter(str.isdigit, str(datetime.now().date())))

# 관측공 별 좌표(X, Y)
spots = {'02' : ['80', '138'], '04' : ['72', '139'], '06' : ['73', '124'], '07' : ['59', '114'], '08' : ['65', '238'], 
         '10' : ['92', '82'], '11' : ['76', '80'], '12' : ['82', '92'], '14' : ['86', '102'], '22' : ['49', '104'], 
         '24' : ['85', '82'], '25' : ['85', '82']}

# 필요한 데이터 컬럼을 가진 빈 데이터 프레임
new_df = pd.DataFrame(columns=['spot', 'fcstDate', 'fcstTime', 'fcstValue'])

# 관측공별 데이터 추출
for spot in spots.keys():
    # X값, Y값
    X = spots[spot][0]
    Y = spots[spot][1]
    # 임시 데이터 프레임
    # temp = pd.DataFrame(columns=['spot', 'fcstDate', 'fcstTime', 'fcstValue'])

    # 필요 파라미터
    params = {'serviceKey' : '',
             'pageNo' : '1', 'numOfRows' : '1000', 'dataType' : 'json', 
             'base_date' : b_date,
             'base_time' : '0500',
             'nx' : X, 'ny' : Y}

    # 불러오기
    response = requests.get(url, params=params)

    # 필요한 데이터가 있는 곳까지 파싱
    aa = json.loads(response.content)["response"]["body"]["items"]["item"]

    for i in range(len(aa)):
        # 오늘 날짜 데이터는 패스
        if aa[i]['fcstDate'] == b_date:
            pass
        else:
            temp_dict = {}
            # temp_list = []
            # 강수량 카테고리
            if aa[i]['category'] == 'PCP':
                # 딕셔너리 형태로 만들어준다.
                temp_dict['spot'] = spot
                temp_dict['fcstDate'] = aa[i]['fcstDate']
                temp_dict['fcstTime'] = aa[i]['fcstTime']

                # 강수량 범주별 데이터 치환
                if (aa[i]['fcstValue'] == '강수없음'):
                    temp_dict['fcstValue'] = 0.0
                elif aa[i]['fcstValue'] == '30.0~50.0mm':
                    temp_dict['fcstValue'] = 30.0
                elif aa[i]['fcstValue'] == '50.0mm 이상':
                    temp_dict['fcstValue'] = 50.0
                else:
                    temp_dict['fcstValue'] = float(aa[i]['fcstValue'][:-2])
                # 데이터 프레임에 딕셔너리 형태의 데이터 행 추가
                new_df = new_df.append(temp_dict, ignore_index=True)
# temp.groupby(['spot', 'fcstDate'])

In [ ]:
##### 테스트용 일 변환 데이터 프레임
# 임시 데이터 프레임
temp = pd.DataFrame(columns=['spot', 'fcstDate', 'fcstValue'])
king_dict = {}
# 관측공과 날짜로 그룹을 만들고 일 강수량으로 변환 
for e,f in new_df.groupby(['spot', 'fcstDate']):
    king_dict['spot'] = e[0]
    king_dict['fcstDate'] = pd.to_datetime(e[1])
    king_dict['fcstValue'] = sum(f['fcstValue'])
    temp = temp.append(king_dict, ignore_index=True)

In [ ]:
temp

,spot,fcstDate,fcstValue
0,02,2022-02-15,0.0
1,02,2022-02-16,0.0
2,02,2022-02-17,0.0
3,04,2022-02-15,0.0
4,04,2022-02-16,0.0
5,04,2022-02-17,0.0
6,06,2022-02-15,0.0
7,06,2022-02-16,0.0
8,06,2022-02-17,0.0
9,07,2022-02-15,0.0
